<a href="https://colab.research.google.com/github/malcolmRite/House-Price-Predictions/blob/master/SGDRegressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [ ]:
# importing necessary libraries 
from sklearn import datasets 
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import log_loss

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pymc3 as pm

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# k-nearest neighbors for multioutput regression

from sklearn.neighbors import KNeighborsRegressor

# linear regression for multioutput regression
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor

# example of evaluating chained multioutput regression with an SVM model
from numpy import mean
from numpy import std
from numpy import absolute
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import RegressorChain
from sklearn.multioutput import ClassifierChain
from sklearn.svm import LinearSVR
from sklearn.svm import SVR

#Importing the graphing libraries
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.linear_model import SGDRegressor

from sklearn.preprocessing import PolynomialFeatures

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# training a Naive Bayes classifier 
from sklearn.naive_bayes import GaussianNB 
from sklearn.naive_bayes import ComplementNB 
from sklearn.metrics import confusion_matrix, accuracy_score
#gnb = GaussianNB().fit(X_train, Y_train) 
#gnb_predictions = gnb.predict(X_test) 

# Importing Training and Test Data

In [ ]:
trainpath = '/content/gdrive/My Drive/Train.csv'
testpath = '/content/gdrive/My Drive/Test.csv'
traindata = pd.read_csv(trainpath)
testdata = pd.read_csv(testpath)

In [ ]:
testdata.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,F86J5PC,1/12/2018,M,M,1984,94KC,DZRV,90QI,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,H6141K3,1/10/2019,M,M,1996,1X1H,J9SY,90QI,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
2,RBAYUXZ,1/1/2020,F,W,1968,UAOD,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
3,KCBILBQ,1/2/2019,M,M,1989,94KC,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,LSEC1ZJ,1/2/2020,F,M,1982,UAOD,0KID,T4MS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


# Data Cleaning

In [ ]:
#Clean Training Data

dtr = traindata 
dtr = dtr.drop('ID', axis=1)
dtr.sex = dtr.sex.replace(['M', 'F'], [1, 0])
dtr = dtr.dropna() #2 entries removed 
dtr.marital_status = dtr.marital_status.replace(['M', 'U', 'S', 'W', 'D', 'P', 'R','f', 'F'], [1,2,3,4,5,6,7,8,8])

dtr.join_date = pd.to_datetime(dtr.join_date, dayfirst=True)
dtr['join_year'] = dtr.join_date.dt.year.astype(int)
dtr['join_month'] = dtr.join_date.dt.month.astype(int)

In [ ]:
#branch code to numeric

tr_branchcode = dtr.branch_code.unique().tolist()
te_branchcode = testdata.branch_code.unique().tolist()
dict_branchcode = dict(zip(tr_branchcode, list(range(1, 16))))
dict_no = 15
for i in range(len(te_branchcode)): 
  if te_branchcode[i] not in dict_branchcode:
    dict_branchcode[te_branchcode[i]] = dict_no + 1
    dict_no = dict_no + 1

#still 15 values therefore no new codes in test data

dtr.branch_code = dtr.branch_code.map(dict_branchcode)

In [ ]:
#occupation code to numeric

tr_occucode = dtr.occupation_code.unique().tolist()
te_occucode = testdata.occupation_code.unique().tolist()
dict_occucode = dict(zip(tr_occucode, list(range(1, 234))))
dict_no = 233
for i in range(len(te_occucode)): 
  if te_occucode[i] not in dict_occucode:
    dict_occucode[te_occucode[i]] = dict_no + 1
    dict_no = dict_no + 1

#242 values therefore 9 more codes seen in test data

dtr.occupation_code = dtr.occupation_code.map(dict_occucode)

#occupation category code to numeric
# 6 codes, no new codes in test data
dict_ocatcode = dict(zip(['T4MS', '90QI', '56SI', 'JD7X', 'L44T', 'AHH5'], [1,2,3,4,5,6]))
dtr.occupation_category_code = dtr.occupation_category_code.map(dict_ocatcode)

In [ ]:
#Clean Test Data 

dte = testdata
#dte = dte.drop('ID', axis=1)
dte.sex = dte.sex.replace(['M', 'F'], [1, 0])
dte.marital_status = dte.marital_status.replace(['M', 'U', 'S', 'W', 'D', 'P', 'R','f', 'F'], [1,2,3,4,5,6,7,8,8])
dte.branch_code = dte.branch_code.map(dict_branchcode)
dte.occupation_code = dte.occupation_code.map(dict_occucode)
dte.occupation_category_code = dte.occupation_category_code.map(dict_ocatcode)
dte.join_date = pd.to_datetime(dte.join_date, dayfirst=True, errors='coerce')
dte['join_year'] = dte.join_date.dt.year #.astype(int)
dte['join_month'] = dte.join_date.dt.month #.astype(int)

#### NA values in test data 

In [ ]:
#Indepedent and Dependent Variables 

features = ['join_year', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code'] 
x_train = dtr[features].copy()
y_train = dtr.iloc[:,7:-2]

#convert to age and years since joining
x_train[['birth_year', 'join_year']] = x_train[['birth_year', 'join_year']].apply(lambda x: 2020.0-x, axis=1)


In [ ]:
x_train = x_train.fillna(2019.0)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size = 0.2, random_state=1)

#test set
dte_s = dte[features + list(y_train.columns)].copy()
#convert to age and years since joining
dte_s[['birth_year', 'join_year']] = dte[['birth_year', 'join_year']].apply(lambda x: 2020-x, axis=1)
test_full = dte_s.copy()
test_full2 = dte_s.copy()

# Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [ ]:
#na date value to mode
dte_s[features] = dte_s[features].fillna(value = 2019)

In [ ]:
X_Pred = sc.fit_transform(dte_s[features])

In [ ]:
#Grade ~ failures + higher_edu + mother_edu + studytime + father_edu + absences

In [ ]:
X_train

,join_year,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code
11494,1.0,1,1,40.0,10,8,3
3682,3.0,0,1,34.0,11,1,1
4207,1.0,1,1,38.0,1,1,1
16872,4.0,0,1,44.0,3,108,1
415,1.0,1,1,34.0,7,1,1
...,...,...,...,...,...,...,...
10955,1.0,1,1,29.0,3,3,3
17290,1.0,1,1,42.0,3,1,1
5192,3.0,0,5,56.0,5,12,2
12172,1.0,1,1,22.0,2,6,1


# Testing Each Parameter

In [ ]:
model = SGDRegressor()
wrapper = MultiOutputRegressor(model)
param_grid = {
    'estimator__alpha': 10.0 ** -np.arange(1, 7),
    'estimator__loss': ['squared_loss', 'huber', 'epsilon_insensitive'],
    'estimator__penalty': ['l2', 'l1', 'elasticnet'],
    'estimator__learning_rate': ['constant', 'optimal', 'invscaling'],
}
clf = GridSearchCV(wrapper, param_grid)
clf.fit(X_train, Y_train)
print("Best score: " + str(clf.best_score_))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1187: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1187: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1187: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1187: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linea

array([1.e-01, 1.e-02, 1.e-03, 1.e-04, 1.e-05, 1.e-06])

# Training Model

In [ ]:
# define base model
model = SGDRegressor(max_iter=50, tol=1e-3, loss='huber')

# define the direct multioutput wrapper model
wrapper = MultiOutputRegressor(model)


# fit the model on the whole dataset
wrapper.fit(X_train, Y_train)

MultiOutputRegressor(estimator=SGDRegressor(alpha=0.0001, average=False,
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.01, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='invscaling',
                                            loss='huber', max_iter=50,
                                            n_iter_no_change=5, penalty='l2',
                                            power_t=0.25, random_state=None,
                                            shuffle=True, tol=0.001,
                                            validation_fraction=0.1, verbose=0,
                                            warm_start=False),
                     n_jobs=None)

In [ ]:
'''# define the evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(wrapper, X_train, Y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=1)
# force the scores to be positive
n_scores = absolute(n_scores)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))'''

MAE: 0.054 (0.001)


In [ ]:
# make a prediction
yhat = wrapper.predict(X_test)


# accuracy on X_test 
accuracy = wrapper.score(X_test, Y_test) 
print(absolute(accuracy))



0.024934123686925963


In [ ]:
yhat = wrapper.predict(X_Pred)

In [ ]:
yhat[1]

array([-6.48645972e-05,  2.47840881e-02,  2.56261864e-03,  4.20076047e-03,
        2.66931134e-03, -4.91020847e-08,  4.10516108e-03,  9.75671873e-01,
        1.34340127e-02,  3.84298790e-03,  5.46862193e-04, -5.67943195e-04,
        1.69893708e-03,  5.93253749e-04,  8.51858963e-01,  1.09169964e-02,
        1.52146552e-04,  3.82586557e-03,  3.33635881e-03,  1.38474477e-03,
        2.53334363e-04])

# Preparing Predicted Values for Submission

In [ ]:
#SUBMISSION
pred1 = pd.DataFrame(yhat)

In [ ]:
pred1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,0.001252,0.000709,-0.000383,0.001850,0.000167,0.000012,-0.000095,0.974901,0.017403,0.000623,0.008980,0.004336,0.002386,0.001329,0.927740,0.048451,0.004737,0.009578,0.008754,0.002825,0.008683
1,-0.000569,0.003819,0.000704,0.000629,0.000103,-0.000003,0.000716,1.001474,0.003789,0.000619,-0.005787,-0.003759,0.000619,0.000350,0.968603,0.008473,0.001095,0.003909,0.004440,0.003134,0.001901
2,-0.000337,0.021124,0.001319,0.002919,0.003334,0.000080,0.003317,0.975628,0.008724,0.003410,0.000245,0.001083,-0.000304,-0.000666,0.901482,0.062054,0.000036,0.003742,0.003997,0.003289,0.004637
3,0.000758,0.003255,0.000460,-0.000402,0.000820,0.000041,0.001568,0.981372,0.008336,0.001243,0.010356,0.006796,0.000807,0.000773,0.940894,0.007361,0.001368,0.006111,0.005099,0.003307,0.005663
4,-0.000546,0.003399,0.000596,0.001123,0.000653,-0.000025,0.000317,0.987058,0.006918,0.001616,-0.002355,-0.001440,0.000176,0.000402,0.952039,0.029706,0.000396,0.007759,0.007254,0.006334,0.001009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.000412,0.002860,0.000693,0.000885,0.000619,-0.000024,0.000512,0.987151,0.007123,0.001840,-0.001291,-0.000584,0.000078,0.000433,0.951168,0.019283,0.000379,0.007782,0.007194,0.006047,0.000864
9996,-0.000540,0.009274,0.000758,0.001077,0.001046,0.000025,0.001187,0.988990,0.005279,0.001080,-0.000792,-0.000917,0.000781,0.000485,0.938479,0.034816,0.000531,0.004215,0.004311,0.004321,0.003382
9997,0.000129,0.003178,0.000461,0.000403,0.000031,-0.000008,0.000894,1.001542,0.003134,0.000274,-0.003249,-0.001914,0.000525,0.000057,0.975282,0.006822,0.001149,0.002148,0.002216,0.001954,0.002078
9998,0.001111,0.008046,0.000754,0.001247,0.001828,0.000031,0.002149,0.968054,0.013766,0.003206,0.012050,0.008250,0.000283,0.000463,0.905857,0.022447,0.001305,0.007934,0.006405,0.003993,0.004896


In [ ]:

vect_pred = [absolute(round(item,5)) for i,row in pred1.iterrows() for item in row]
vect_pred

[0.00125,
 0.00071,
 0.00038,
 0.00185,
 0.00017,
 1e-05,
 0.0001,
 0.9749,
 0.0174,
 0.00062,
 0.00898,
 0.00434,
 0.00239,
 0.00133,
 0.92774,
 0.04845,
 0.00474,
 0.00958,
 0.00875,
 0.00283,
 0.00868,
 0.00057,
 0.00382,
 0.0007,
 0.00063,
 0.0001,
 0.0,
 0.00072,
 1.00147,
 0.00379,
 0.00062,
 0.00579,
 0.00376,
 0.00062,
 0.00035,
 0.9686,
 0.00847,
 0.00109,
 0.00391,
 0.00444,
 0.00313,
 0.0019,
 0.00034,
 0.02112,
 0.00132,
 0.00292,
 0.00333,
 8e-05,
 0.00332,
 0.97563,
 0.00872,
 0.00341,
 0.00025,
 0.00108,
 0.0003,
 0.00067,
 0.90148,
 0.06205,
 4e-05,
 0.00374,
 0.004,
 0.00329,
 0.00464,
 0.00076,
 0.00325,
 0.00046,
 0.0004,
 0.00082,
 4e-05,
 0.00157,
 0.98137,
 0.00834,
 0.00124,
 0.01036,
 0.0068,
 0.00081,
 0.00077,
 0.94089,
 0.00736,
 0.00137,
 0.00611,
 0.0051,
 0.00331,
 0.00566,
 0.00055,
 0.0034,
 0.0006,
 0.00112,
 0.00065,
 3e-05,
 0.00032,
 0.98706,
 0.00692,
 0.00162,
 0.00236,
 0.00144,
 0.00018,
 0.0004,
 0.95204,
 0.02971,
 0.0004,
 0.00776,
 0.00725,
 

In [ ]:
# Format and create the submission file out of the predicted probabilites 

def submit_format(test_data, pred_vect):
  
  # The first row
  submit_dict = {'ID X PCODE': 'Label'}
  i = 0

  # iterate for each ID over each product
  for id in test_data.ID:
    for product in test_data.columns[8:29]:
      
      submit_dict['{} X {}'.format(id, product)] = pred_vect[i]
        
      i += 1

  submit = pd.Series(submit_dict)
  submit_file = submit.to_csv('/content/gdrive/My Drive/submission.csv', header=False)

In [ ]:
dte.columns

Index(['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3',
       'join_year', 'join_month'],
      dtype='object')

In [ ]:
submit_format(dte, vect_pred)

In [ ]:
testdata

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3,join_year,join_month
0,F86J5PC,2018-12-01,1,1,1984,10,114,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2018.0,12.0
1,H6141K3,2019-10-01,1,1,1996,1,21,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,2019.0,10.0
2,RBAYUXZ,2020-01-01,0,4,1968,2,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,2020.0,1.0
3,KCBILBQ,2019-02-01,1,1,1989,10,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2019.0,2.0
4,LSEC1ZJ,2020-02-01,0,1,1982,2,7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2020.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0GMU5UH,2019-04-01,0,1,1986,3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,2019.0,4.0
9996,I9W11CD,2019-08-01,1,1,1980,2,7,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2019.0,8.0
9997,42WTEGT,2018-04-01,1,1,1993,3,8,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2018.0,4.0
9998,8EKC4O9,2015-05-01,0,2,1981,11,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2015.0,5.0
